# Correr streamlit app desde Google Colab Notebook

In [102]:
!pip install -q streamlit

# Proyecto Final: Recordatorio y Envío de Felicitaciones de Cumpleaños


El proyecto esta hecho en python con la idea de realizar el front en streamlit, viene explicada cada parte del codigo para un mayor entendimiento del codigo.

In [98]:
%%writefile app.py
import streamlit as st
from datetime import datetime, timedelta
import pickle
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Clase para manejar los cumpleaños
class BirthdayManager:
    def __init__(self, storage_file="birthdays.pkl"):
        self.storage_file = storage_file
        self.birthday_data = self.load_birthdays()

    def load_birthdays(self):
        # Intentamos cargar los datos desde un archivo pickle
        if os.path.exists(self.storage_file):
            with open(self.storage_file, "rb") as file:
                birthday_data = pickle.load(file)
                # Aseguramos que cada entrada tenga la clave 'message' y 'felicitacion_enviada'
                for data in birthday_data:
                    if 'message' not in data:
                        data['message'] = "¡Feliz Cumpleaños! Que tengas un día maravilloso."
                    if 'felicitacion_enviada' not in data:
                        data['felicitacion_enviada'] = False  # Inicializamos como False
                return birthday_data
        else:
            return []

    def save_birthdays(self):
        # Guardamos los datos en el archivo pickle
        with open(self.storage_file, "wb") as file:
            pickle.dump(self.birthday_data, file)

    def add_birthday(self, name, email, month, day, message):
        # Asegurarnos de que todos los campos estén presentes
        if not message or message.strip() == "":
            message = "¡Feliz Cumpleaños! Que tengas un día maravilloso."

        # Agregar un nuevo cumpleaños a la lista (sin considerar el año)
        self.birthday_data.append({
            "name": name,
            "email": email,
            "month": month,
            "day": day,
            "message": message,  # Guardamos el mensaje
            "felicitacion_enviada": False  # Inicializamos como False
        })
        self.save_birthdays()

        # Verificar si el cumpleaños es hoy y enviar la felicitación si es así
        today = datetime.today().date()
        next_birthday = datetime(today.year, month, day).date()
        if today == next_birthday:
            self.send_birthday_email(email, message)
            # Marcamos que la felicitación ya fue enviada
            self.birthday_data[-1]["felicitacion_enviada"] = True
            self.save_birthdays()

    def get_upcoming_birthdays(self):
        # Calculamos cuántos días faltan para el próximo cumpleaños
        upcoming = []
        today = datetime.today().date()
        for data in self.birthday_data:
            # Creamos una fecha solo con el mes y día, el año será el actual
            next_birthday = datetime(today.year, data["month"], data["day"]).date()

            # Si el cumpleaños ya pasó este año, lo ajustamos al siguiente
            if next_birthday < today:
                next_birthday = next_birthday.replace(year=today.year + 1)

            days_left = (next_birthday - today).days
            upcoming.append({
                "name": data["name"],
                "email": data["email"],
                "days_left": days_left,
                "next_birthday": next_birthday,
                "message": data["message"],
                "felicitacion_enviada": data["felicitacion_enviada"]
            })

        # Ordenamos por los próximos cumpleaños
        upcoming.sort(key=lambda x: x["days_left"])
        return upcoming

    def send_birthday_email(self, recipient_email, message, subject="¡Feliz Cumpleaños!"):
        # Función para enviar correo electrónico
        usuario = "poo.proyecto.220661135@gmail.com"  # Cambia esto por tu correo real
        contraseña = "nffg bwjg hhmq pdwt"  # Cambia esto por tu contraseña real (si usas contraseñas de aplicación)
        servidor_smtp = "smtp.gmail.com"
        puerto_smtp = 587  # Usamos 587 para TLS

        try:
            # Crear el mensaje
            msg = MIMEMultipart()
            msg["From"] = usuario
            msg["To"] = recipient_email
            msg["Subject"] = subject

            # Codificar el mensaje
            msg.attach(MIMEText(message, "plain", _charset="utf-8"))

            # Establecer conexión con el servidor SMTP
            with smtplib.SMTP(servidor_smtp, puerto_smtp) as server:
                server.starttls()  # Usar TLS para asegurar la conexión
                server.login(usuario, contraseña)  # Iniciar sesión en el servidor
                server.sendmail(usuario, recipient_email, msg.as_string())  # Enviar el mensaje

            print(f"Correo de felicitación enviado a {recipient_email}")

        except Exception as e:
            print(f"Error al enviar el correo: {e}")

    def check_and_send_birthday_greetings(self):
        # Verifica si hoy es el cumpleaños de alguien y envía el mensaje solo si no se ha enviado antes
        today = datetime.today().date()
        for data in self.birthday_data:
            next_birthday = datetime(today.year, data["month"], data["day"]).date()

            if today == next_birthday and not data["felicitacion_enviada"]:
                # Verificamos si la clave 'message' existe antes de acceder a ella
                message = data.get("message", "¡Feliz Cumpleaños! Que tengas un día maravilloso.")
                self.send_birthday_email(data["email"], message)
                # Marcamos que la felicitación ya fue enviada
                data["felicitacion_enviada"] = True
                self.save_birthdays()

    def remove_birthday(self, email):
        # Eliminar un cumpleaños basado en el correo electrónico
        self.birthday_data = [data for data in self.birthday_data if data["email"] != email]
        self.save_birthdays()

# Funciones de Streamlit
def app():
    st.title("Recordatorios de cumpleaños.")
    st.write("Proyecto Final: Recordatorio y Envío de Felicitaciones de Cumpleaños realizado por Jared Méndez García")

    # Crear instancia de BirthdayManager
    manager = BirthdayManager()

    # Verificar y enviar felicitaciones si es el cumpleaños hoy
    manager.check_and_send_birthday_greetings()

    # Opción para registrar cumpleaños
    st.header("Registrar Cumpleaños")
    name = st.text_input("Nombre:")
    email = st.text_input("Correo Electrónico:")
    month = st.number_input("Mes de Nacimiento:", min_value=1, max_value=12, step=1)
    day = st.number_input("Día de Nacimiento:", min_value=1, max_value=31, step=1)

    # Lista de mensajes predefinidos
    default_messages = [
        "¡Feliz Cumpleaños! Que tengas un día maravilloso.",
        "¡Muchas felicidades! Espero que pases un excelente día.",
        "¡Felicidades en tu día! Que este nuevo año de vida esté lleno de bendiciones."
    ]

    # Opción para elegir mensaje predefinido o escribir uno personalizado
    message_choice = st.radio("Elige un mensaje:", ["Mensaje Personalizado", "Mensaje Predeterminado"])

    if message_choice == "Mensaje Predeterminado":
        birthday_message = st.selectbox("Selecciona un mensaje:", default_messages)
    else:
        birthday_message = st.text_area("Escribe tu mensaje personalizado:")

    if st.button("Registrar Cumpleaños"):
        if name and email and month and day:
            manager.add_birthday(name, email, month, day, birthday_message)
            st.success(f"Cumpleaños de {name} registrado correctamente.")
        else:
            st.error("Por favor, ingresa todos los campos.")

    # Mostrar los cumpleaños próximos
    st.header("Cumpleaños Próximos")
    upcoming_birthdays = manager.get_upcoming_birthdays()
    if upcoming_birthdays:
        for birthday in upcoming_birthdays:
            st.write(f"**{birthday['name']}** - {birthday['next_birthday']} - Faltan {birthday['days_left']} días")
            st.write(f"Mensaje a enviar: {birthday['message']}")
            st.write(f"Correo: {birthday['email']}")
            # Opción para eliminar un cumpleaños
            if st.button(f"Eliminar {birthday['name']}"):
                manager.remove_birthday(birthday['email'])
                st.success(f"Cumpleaños de {birthday['name']} eliminado correctamente.")
    else:
        st.write("No hay cumpleaños registrados.")

if __name__ == "__main__":
    app()


Overwriting app.py


# Instalar tunel local

In [99]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

## Correr streamlit en el fondo

In [100]:
!streamlit run /content/app.py &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [101]:
!npx localtunnel --port 8501

⠙your url is: https://weak-results-speak.loca.lt
^C


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)